In [2]:
import numpy as np
from typing import Callable
from numpy.typing import NDArray, ArrayLike
import csv
from datetime import datetime, timezone
import astropy
import pytz
from astropy.time import Time
import math
from pyquaternion import Quaternion
import pymap3d
import matplotlib.pyplot as plt
import ppigrf

In [3]:
def raw_igdf_enu(height_from_sea_level : np.float64, lat : np.float64, lon : np.float64, time):
    #kilometers, degrees, degrees
    #returns B_east, b_north, b_up in nT: https://github.com/IAGA-VMOD/ppigrf/tree/main
    return ppigrf.igrf(lon, lat, height_from_sea_level, time)

def igdf_eci_vector(x : np.float64, y : np.float64, z : np.float64, time):
    #eci in METERS, returns unit eci igdf vector
    lat, lon, alt = pymap3d.eci2geodetic(x, y, z, time)
    b_east, b_north, b_up = raw_igdf_enu(alt/1000, lat, lon, time)
    ecefb_x, ecefb_y, ecefb_z = pymap3d.enu2ecef(b_east, b_north, b_up, lat, lon, alt)
    b_x, b_y, b_z = pymap3d.ecef2eci(ecefb_x, ecefb_y, ecefb_z, time)
    b_vec = np.array([b_x - x, b_y - y, b_z - z])
    return b_vec

In [15]:
Q = np.eye(6)#use the Q equation from Table II of the "backup satellite" paper
R = np.eye(3)*10#use Equation 26 when actually finding R
dt = 10
def xi(quaternion, angular_velocities):
    angular_velocity_quat = Quaternion(scalar = 0, vector = angular_velocities*dt)
    return 1/2*quaternion*angular_velocity_quat
    
def cross_product_matrix(mat):
    return np.cross(np.eye(3), mat)

#note that, in all cases, if no measurement is received, we simply must dead reckon, tough luck
def MEKF(previous_state, previous_covariance, measured_quaternion):
    #measured quaternion received from QUEST based either on sun_sensor + magnetometer OR bdot + magnetometer
    H = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]
    ])#the h function is assumed to simply return the angular component of the quaternion -- hence, the shape
    
    F = np.concatenate([
        np.concatenate([-cross_product_matrix(previous_state[4:]), -np.eye(3)], axis = 1),
        np.zeros((3,6))
    ])
    #smart people found this based on the update function f(quaternion, w) = quaternion + xi(quaternion, w)
    #i am not smart people

    previous_quaternion = Quaternion(previous_state[:4])
    angular_velocities = previous_state[4:]
    estimated_quaternion = previous_quaternion + xi(previous_quaternion, angular_velocities)#estimate based on derivative

    #standard kalman filter stuff
    estimated_cov_diff = F@previous_covariance@F.T + Q
    S = H@estimated_cov_diff@H.T + R
    kalman_gain = estimated_cov_diff@H.T@np.linalg.inv(S)
    predicted_covariance = (np.eye(6) - kalman_gain@H)@estimated_cov_diff

    #multiplicative EKF specific calculations
    error_quaternion = measured_quaternion*(previous_quaternion.inverse)
    a = error_quaternion.elements[1:]/error_quaternion[0]
    d_state = kalman_gain@a
    new_angular_velocities = angular_velocities + d_state[3:]
    d_attitude = Quaternion(vector = d_state[:3], scalar = 2)#gives the attitude error

    #applying d_attitude to the previous quaternion and integrating
    d_quat = d_attitude* previous_quaternion + xi(d_attitude* previous_quaternion, angular_velocities)
    predicted_quaternion_unnormal = estimated_quaternion + d_quat
    
    predicted_quaternion_normal = predicted_quaternion_unnormal/predicted_quaternion_unnormal.norm

    predicted_state = np.concatenate([predicted_quaternion_normal.elements, new_angular_velocities], axis = 0)
    return predicted_state, predicted_covariance
    

In [27]:
starting_state = np.array([1, 0, 0, 0, 0.001, 0.001, 0.001])
measured_quaternion = Quaternion([0.796, 0.196, 0.304, 0.486])
cov = np.eye(6) * 10000
for i in range(1000):
    print(starting_state)
    starting_state, cov = MEKF(starting_state, cov, measured_quaternion)


[1.    0.    0.    0.    0.001 0.001 0.001]
[0.96731651 0.08395667 0.12870827 0.20170333 0.001      0.001
 0.001     ]
[0.95891195 0.0954608  0.14525016 0.22422646 0.001      0.001
 0.001     ]
[0.95019316 0.10611164 0.16065815 0.24507599 0.001      0.001
 0.001     ]
[0.94127892 0.11598094 0.17502606 0.26440178 0.001      0.001
 0.001     ]
[0.93226587 0.12513277 0.18843716 0.28233593 0.001      0.001
 0.001     ]
[0.92323272 0.13362467 0.20096594 0.29899578 0.001      0.001
 0.001     ]
[0.91424355 0.14150856 0.21267938 0.31448615 0.001      0.001
 0.001     ]
[0.90535034 0.1488315  0.22363805 0.32890116 0.001      0.001
 0.001     ]
[0.89659505 0.1556363  0.23389695 0.34232567 0.001      0.001
 0.001     ]
[0.88801132 0.16196203 0.24350619 0.35483648 0.001      0.001
 0.001     ]
[0.87962577 0.16784439 0.25251159 0.36650328 0.001      0.001
 0.001     ]
[0.87145917 0.17331614 0.26095513 0.37738953 0.001      0.001
 0.001     ]
[0.86352732 0.17840734 0.26887539 0.3875531  0.001      

In [26]:
qd = Quaternion(starting_state[:4]).conjugate * measured_quaternion
phi   = math.atan2( 2 * (qd.w * qd.x + qd.y * qd.z), 1 - 2 * (qd.x**2 + qd.y**2) )
theta = math.asin ( 2 * (qd.w * qd.y - qd.z * qd.x) )
psi   = math.atan2( 2 * (qd.w * qd.z + qd.x * qd.y), 1 - 2 * (qd.y**2 + qd.z**2) )
print(phi, theta, psi)

-0.1637673804796534 -0.1928619953861062 -0.16376764476085592


In [315]:
print(starting_state)
print(measured_quaternion)

[0.98667575 0.11524101 0.11483157 0.00204623 0.001      0.001
 0.001     ]
0.985 +0.123i +0.123j +0.000k


In [239]:
np.cross([1,2,3],[1,2,3])

array([0, 0, 0])

In [335]:
def xi(quaternion, angular_velocities):
    angular_velocity_quat = Quaternion(scalar = 0, vector = angular_velocities)
    return 1/2*quaternion*angular_velocity_quat

In [378]:
Gamma = np.array(
        [[-1, 0, 0, 0, 0, 0],
        [0, -1, 0, 0, 0, 0],
        [0, 0, -1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]
    )

test = np.array(
        [[1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4],
        [0, 0, 0, 0, 5, 0],
        [0, 0, 0, 0, 0, 6]]
    )
print(Gamma@test@Gamma.T)

[[ 1  1  1 -1 -1 -1]
 [ 2  2  2 -2 -2 -2]
 [ 3  3  3 -3 -3 -3]
 [-4 -4 -4  4  4  4]
 [ 0  0  0  0  5  0]
 [ 0  0  0  0  0  6]]
